# Configuraciones y librerias

In [ ]:
# Librerias
import geopandas as gpd
import rioxarray as rxr
import xarray as xr
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import threading
from owslib.wfs import WebFeatureService

# Fechas de interes
start_date = "2022-01-01"
end_date = "2022-12-31"
url_adef_intg = (
    "https://data-api.globalforestwatch.org/dataset/gfw_integrated_alerts/"
    "latest/download/geotiff?grid=10/100000&tile_id=20N_090W&pixel_meaning="
    "date_conf&x-api-key=2d60cd88-8348-4c0f-a6d5-bd9adb585a8c"
)

# Preparar la data para análisis

## Preparar los datos auxiliares

In [ ]:
# Crear la conexion al servicio WFS
url_icf_wfs = "https://geoserver.icf.gob.hn/icfpub/wfs"
wfs_icf = WebFeatureService(url_icf_wfs, version="1.1.0")

# Obtenemos la lista de capas disponibles
layers = wfs_icf.contents
layers

In [ ]:
# Layers de interes

## Leer y filtrar el tif por las fechas de interes

In [ ]:
# Identificar el tiempo de análisis
zero_day = pd.Timestamp("2014-12-31")
days_to_start = (pd.Timestamp(start_date) - zero_day).days
days_to_end = (pd.Timestamp(end_date) - zero_day).days

# Leer el tif de alertas
tif_adef_intg = rxr.open_rasterio(url_adef_intg, lock=True, chunks=True)

In [ ]:
tif_days = tif_adef_intg % 10000
mask = (tif_days >= days_to_start) & (tif_days <= days_to_end)

tif_filtered = tif_adef_intg.where(mask, drop=True)

In [ ]:
print(f"Days to start: {days_to_start}")
print(f"Days to end: {days_to_end}")